In [13]:
import os
import pg8000
from dotenv import load_dotenv
import os
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import subprocess
import os
from tqdm import tqdm




In [3]:
# Load environment variables from .env file
load_dotenv()

# Database connection details
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")


In [4]:
engine = create_engine('postgresql+pg8000://'+DB_USER+':'+DB_PASSWORD+'@'+DB_HOST+':5432/'+DB_NAME)

In [5]:
Querry_string = "SELECT storage, fileid, path, path_hash, name, etag FROM public.oc_filecache WHERE path like 'files/%' and (path LIKE '%.JPG' OR path LIKE '%.jpg') and storage = 1 ORDER BY fileid DESC LIMIT 10000"

In [6]:
try:
	df_file_loc = pd.read_sql_query(sqlalchemy.text(Querry_string), engine)
	print("done")
except Exception as e:
	print("Error:", e)

done


In [7]:
Querry_string = "SELECT * FROM public.oc_storages ORDER BY numeric_id ASC LIMIT 100"

In [8]:
try:
	df_user_loc = pd.read_sql_query(sqlalchemy.text(Querry_string), engine)
	print("done")
except Exception as e:
	print("Error:", e)

done


In [9]:
df_user_loc['id_clean'] = df_user_loc['id'].str.replace('(home::|local::)', '', regex=True)

In [10]:
df_file_loc = pd.merge(df_file_loc, df_user_loc[['numeric_id', 'id_clean']], 
          left_on='storage', 
          right_on='numeric_id', 
          how='left')

In [11]:
df_file_loc['full_path'] = df_file_loc['id_clean'].astype(str) + '/' + df_file_loc['path'].astype(str)

In [27]:
# Get first 10 paths from dataframe
paths = df_file_loc['full_path'].tail(10).tolist()

# Base remote path
remote_base = os.getenv("REMOTE_BASE_PATH")

# password and user
password = os.getenv("SSHPASS")
ssh_user = os.getenv("SSHUSER")
ssh_ip = os.getenv("SSHIP")

# Create a cache directory if it doesn't exist
cache_dir = 'image_cache'
os.makedirs(cache_dir, exist_ok=True)
print(paths)
# SSH command to list files
for path in tqdm(paths):
    # Clean the path (remove 'files/' prefix)
    full_path = f"{remote_base}{path}"
    # Escape spaces in the path for shell commands
    escaped_path = full_path.replace(' ', '\\ ')
    
    # Execute SSH command with sudo and provide password through stdin
    cmd = f'''sshpass -p {password} ssh -o StrictHostKeyChecking=no {ssh_user}@{ssh_ip} "echo {password} | sudo -S ls -l '{full_path}'"'''
    try:
        result = subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)

        # Extract the filename from the path
        filename = os.path.basename(path)
        local_path = os.path.join(cache_dir, filename)

        # Use sudo with SCP to copy the file
        scp_cmd = f'''sshpass -p {password} ssh -o StrictHostKeyChecking=no {ssh_user}@{ssh_ip} "echo {password} | sudo -S cat '{full_path}'" > "{local_path}"'''
        scp_result = subprocess.run(scp_cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)

        if os.path.getsize(local_path) < 1:
            print(f"Error copying file {filename}")
    except Exception as e:
        print(f"Error accessing {full_path}: {str(e)}")


['Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/ausschnitte_aus_der_mitte_des_jahrhunderts.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/ausschnitte_bauhaus_2_kopie.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/ausschnitte_aus_der_mitte_des_jahrhunderts2.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/aqua_shapes4_kopie.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/aqua_shapes3_kopie.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/aqua_shapes2_kopie.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/aquarell_miro_shapes_kopie.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/aqua_shapes1_kopie.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzeln/aquarell_miro_shapes3e_kopie.jpg', 'Tom/files/Bre/Copyright_infringement/2025/KunstKopie/kk jpgs einzel

100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


In [ ]:
#cleanup cache directory
os.listdir(cache_dir)
for file in tqdm(os.listdir(cache_dir)):
    file_path = os.path.join(cache_dir, file)
    if os.path.isfile(file_path):
        os.remove(file_path)

100%|██████████| 20/20 [00:00<00:00, 327.85it/s]
